## 导入包

In [2]:
# Import what we need
import os
import sys
import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf  # Import TensorFlow after Scipy or Scipy will break
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
%matplotlib inline

## 路径

In [3]:
# 输出图片的路径
OUTPUT_DIR = '/home/liuzixuan/实训/02-用Python快速实现图片的风格迁移/output_image'
# 风格图片的路径
STYLE_IMAGE = '/home/liuzixuan/实训/02-用Python快速实现图片的风格迁移/img/star.jpg'
# 内容图片的路径
CONTENT_IMAGE = '/home/liuzixuan/实训/02-用Python快速实现图片的风格迁移/img'
# VGG-19模型的路径
VGG_MODEL = '/home/liuzixuan/实训/02-用Python快速实现图片的风格迁移/刘子璇/imagenet-vgg-verydeep-19.mat'
# 模型存取路径
MODEL = '/home/liuzixuan/实训/02-用Python快速实现图片的风格迁移/刘子璇/model/'

## 所需常数定义

In [4]:
# 输出图片大小定义(224,224,3)
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
COLOR_CHANNELS = 3
# 噪声比(与内容图像混合的噪声权重百分比)
NOISE_RATIO = 0.6
# Constant to put more emphasis on content loss.
BETA = 5
# Constant to put more emphasis on style loss.
ALPHA = 100
# 从VGG模型的输入中减去的平均值
MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
# 训练次数
ITERATIONS = 2000 

STYLE_WEIGHT = 1
CONTENT_WEIGHT = 1
STYLE_LAYERS = ['relu1_2','relu2_2','relu3_2']
CONTENT_LAYERS = ['relu1_2']
_vgg_params = None

## 定义模型函数

In [5]:
def load_vgg_model(path):
    """
    Returns a model for the purpose of 'painting' the picture.
    Takes only the convolution layer weights and wrap using the TensorFlow
    Conv2d, Relu and AveragePooling layer. VGG actually uses maxpool but
    the paper indicates that using AveragePooling yields better results.
    The last few fully connected layers are not used.
    Here is the detailed configuration of the VGG model:
        0 is conv1_1 (3, 3, 3, 64)
        1 is relu
        2 is conv1_2 (3, 3, 64, 64)
        3 is relu    
        4 is maxpool
        5 is conv2_1 (3, 3, 64, 128)
        6 is relu
        7 is conv2_2 (3, 3, 128, 128)
        8 is relu
        9 is maxpool
        10 is conv3_1 (3, 3, 128, 256)
        11 is relu
        12 is conv3_2 (3, 3, 256, 256)
        13 is relu
        14 is conv3_3 (3, 3, 256, 256)
        15 is relu
        16 is conv3_4 (3, 3, 256, 256)
        17 is relu
        18 is maxpool
        19 is conv4_1 (3, 3, 256, 512)
        20 is relu
        21 is conv4_2 (3, 3, 512, 512)
        22 is relu
        23 is conv4_3 (3, 3, 512, 512)
        24 is relu
        25 is conv4_4 (3, 3, 512, 512)
        26 is relu
        27 is maxpool
        28 is conv5_1 (3, 3, 512, 512)
        29 is relu
        30 is conv5_2 (3, 3, 512, 512)
        31 is relu
        32 is conv5_3 (3, 3, 512, 512)
        33 is relu
        34 is conv5_4 (3, 3, 512, 512)
        35 is relu
        36 is maxpool
        37 is fullyconnected (7, 7, 512, 4096)
        38 is relu
        39 is fullyconnected (1, 1, 4096, 4096)
        40 is relu
        41 is fullyconnected (1, 1, 4096, 1000)
        42 is softmax
    """
    vgg = scipy.io.loadmat(path)

    vgg_layers = vgg['layers']
    # 定义权重函数(返回各层权重及偏置值)
    def _weights(layer, expected_layer_name):
        W = vgg_layers[0][layer][0][0][0][0][0]
        b = vgg_layers[0][layer][0][0][0][0][1]
        layer_name = vgg_layers[0][layer][0][0][-2]
        assert layer_name == expected_layer_name
        return W, b
    # 定义各层函数
    def _relu(conv2d_layer):
        return tf.nn.relu(conv2d_layer)

    def _conv2d(prev_layer, layer, layer_name):
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(
            prev_layer, filter=W, strides=[1, 1, 1, 1], padding='SAME') + b

    def _conv2d_relu(prev_layer, layer, layer_name):
        return _relu(_conv2d(prev_layer, layer, layer_name))

    def _avgpool(prev_layer):
        return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 构造图形模型
    graph = {}
    graph['input']   = tf.Variable(np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)), dtype = 'float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    return graph

## 内容损失函数

In [6]:
def content_loss(target_features,content_features):
    # 使用特征图之差的平方和作为内容差距，越小则合成图与原图的内容越接近
    _,height,width,channel = map(lambda i:i.value,content_features.get_shape())
    #print ('content_features.get_shape() : ')
    #print (content_features.get_shape())
    content_size = height * width * channel
    return tf.nn.l2_loss(target_features - content_features) / content_size

## 风格损失函数

In [7]:
def style_loss(target_features,style_features):
    # 使用Gram matrix之差的平方和作为风格差距，越小则合成图像越具有风格图的纹理特征
    _,height,width,channel = map(lambda i:i.value,target_features.get_shape())
    #print ('target_features.get_shape() : ')
    #print (target_features.get_shape())
    size = height * width * channel
    #targer gram 是特征图矩阵的内积
    target_features = tf.reshape(target_features,(-1,channel))
    target_gram = tf.matmul(tf.transpose(target_features),target_features) / size

    style_features = tf.reshape(style_features,(-1,channel))
    style_gram = tf.matmul(tf.transpose(style_features),style_features) / size

    return tf.nn.l2_loss(target_gram - style_gram) / size

## 总损失函数

In [8]:
def loss_function(style_image,content_image,target_image):
    #style_features = vgg19([style_image])
    #content_features = vgg19([content_image])
    #target_features = vgg19([target_image])
    style_features = model['input'].assign(style_image)
    content_features = model['input'].assign(content_image)
    target_features = model['input'].assign(target_image)
    #content_loss = content_loss(target_features=target_features,content_features=content_features)
    #style_loss = style_loss(target_features=target_features,style_features=style_features)
    #loss = 0.0
    #for layer in CONTENT_LAYERS:
    #    loss +=  CONTENT_WEIGHT * content_loss(target_features[layer],content_features[layer])

    #for layer in STYLE_LAYERS:
    #    loss +=  STYLE_WEIGHT * style_loss(target_features[layer],style_features[layer])
    
    loss = BETA * content_loss(target_features,content_features) + ALPHA * style_loss(target_features,style_features)
    
    return loss

In [9]:
# 风格图片
style_image = load_image(STYLE_IMAGE)
#imshow(style_image[0])
    
# 模型加载
model = load_vgg_model(VGG_MODEL)
with tf.Session() as sess:
    style_features = model['input'].assign(style_image)
    sess.run(style_features)
    print(style_features)

NameError: name 'load_image' is not defined

In [13]:
print(style_image.shape)

NameError: name 'style_image' is not defined

## 噪声图像

In [14]:
def generate_noise_image(content_image, noise_ratio = NOISE_RATIO):
    # 噪声图像:与内容图像以一定比例混合
    noise_image = np.random.uniform(
            -20, 20,
            (1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)).astype('float32')
    # 加权平均值
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    return input_image

## 读取图片

In [15]:
def load_image(path):
    image = scipy.misc.imread(path)
    # 统一图片大小(1,224,224,3)
    image = scipy.misc.imresize(image,(224,224,3))
    image = np.reshape(image, ((1,) + image.shape))
    # VGG模型的输入需要减去平均值
    image = image - MEAN_VALUES
    return image

## 图片存储

In [16]:
def save_image(path, image):
    # 输出图片需要加回平均值
    image = image + MEAN_VALUES
    # 去掉第一个无用的维度，剩下的就是图像
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [17]:
with tf.Graph().as_default() as g:
    style = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32)
    content = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32)
    target = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32)
    # 模型加载
    model = load_vgg_model(VGG_MODEL)
    loss = loss_function(style,content,target)
    saver = tf.train.Saver()

In [18]:
with tf.Session(graph=g) as sess:
    # 优化器
    optimizer = tf.train.AdamOptimizer(2.0)
    #rain_op = optimizer.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    style_image = load_image(STYLE_IMAGE)
    content_image = load_image(CONTENT_IMAGE+'/'+'img0'+str(1)+'.jpg')
    target_image = generate_noise_image(content_image)
    print(type(style_image))
    # 训练模型
    for step in range(ITERATIONS):
        #oss,_ = sess.run([loss,train_op],feed_dict={style:style_image,content:content_image,target:target_image})
        loss=sess.run(loss,feed_dict={style:style_image,content:content_image,target:target_image})
        if step % 200 == 0:
            mixed_image = sess.run(model['input'])
            print('step %5d, loss %2.4f' % (step, loss))
            filename = OUTPUT_DIR + '/' + 'img0' + str(step)+'.jpg'
            saver.save(sess, MODEL+'model'+str(step/200))

InternalError: Failed to create session.

## 训练

In [27]:
with tf.Session() as sess:
    #优化器
    #optimizer = tf.train.AdamOptimizer(2.0)
    #train_step = optimizer.minimize(total_loss)
    #sess.run(tf.global_variables_initializer())
    # 训练模型
    
    #for step in range(1000):
    #    #batch_images, batch_labels = mnist.train.next_batch(32)
    #    
    #    res_loss, _ = sess.run([total_loss, train_step], feed_dict={
    #        content: content_image,
    #        style: style_image
    #    })
        
        # 输出代价并验证模型
    #    if step % 100 == 0:
    #        accs = []
    #        for test_step in range(10000 // 32):
    #            batch_images, batch_labels = mnist.test.next_batch(32)
    #            res_acc = sess.run(acc, feed_dict={
    #                inputs: batch_images,
    #                labels: batch_labels
    #            })
    #           accs.append(res_acc)
    #        accs = np.mean(accs)
    #        print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))
    
    
    
    sess.run(tf.global_variables_initializer())
    #saver = tf.train.Saver()
    # 风格图片
    style_image = load_image(STYLE_IMAGE)
    #imshow(style_image[0])

    # 模型加载
    model = load_vgg_model(VGG_MODEL)
    #print(model)

    #print(content_image.shape)
    #print(style_image.shape)
    for i  in range(1,6):
        # 内容图片
        content_image = load_image(CONTENT_IMAGE+'/'+'img0'+str(i)+'.jpg')
        #imshow(content_image[0])
    
        # 输入图片
        input_image = generate_noise_image(content_image)
        #imshow(input_image[0])
    
        # 内容损失
        sess.run(model['input'].assign(content_image))
        content_loss = content_loss_func(sess, model)
    
        # 风格损失
        sess.run(model['input'].assign(style_image))
        style_loss = style_loss_func(sess, model)
    
        #总损失
        total_loss = BETA * content_loss + ALPHA * style_loss
    
        #优化器
        optimizer = tf.train.AdamOptimizer(2.0)
        train_step = optimizer.minimize(total_loss)
        
        sess.run(tf.initialize_all_variables())
        sess.run(model['input'].assign(input_image))
    #    sess.run(tf.initialize_all_variables())
    #    sess.run(model['input'].assign(input_image))
        print('img0',i)
        for j in range(ITERATIONS):
            sess.run(train_step)
            # 每迭代１００次输出一次
            if j%100 == 0:
                #tf.train.Saver.save(sess=sess,save_path=MODEL+'model_'+str(i),global_step=j)
                mixed_image = sess.run(model['input'])
                print('Iteration %d' % (j))
                print('sum : ', sess.run(tf.reduce_sum(mixed_image)))
                print('cost: ', sess.run(total_loss))

                if not os.path.exists(OUTPUT_DIR):
                    os.mkdir(OUTPUT_DIR)

                filename = OUTPUT_DIR + '/' + 'img0'+ str(i)+'_'+ str(j) + '.jpg' 
                save_image(filename, mixed_image)
        print('###########################################################################')
        #saver.save(sess,'model'+str(i))

/home/liuzixuan/test/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/liuzixuan/test/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.


img0 1
Iteration 0
sum :  297664.7
cost:  143579330000.0


/home/liuzixuan/test/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys


Iteration 100
sum :  -40959.406
cost:  1989663400.0
Iteration 200
sum :  -232078.22
cost:  1067706900.0
Iteration 300
sum :  -396636.62
cost:  746889300.0
Iteration 400
sum :  -556374.4
cost:  566964160.0
Iteration 500
sum :  -712374.5
cost:  453271460.0
Iteration 600
sum :  -864302.4
cost:  373953730.0
Iteration 700
sum :  -1008806.2
cost:  318392770.0
Iteration 800
sum :  -1145272.8
cost:  278326660.0
Iteration 900
sum :  -1274023.5
cost:  248263150.0
Iteration 1000
sum :  -1397839.0
cost:  224924670.0
Iteration 1100
sum :  -1515923.8
cost:  209082740.0
Iteration 1200
sum :  -1628693.0
cost:  192290450.0
Iteration 1300
sum :  -1736330.1
cost:  182231790.0
Iteration 1400
sum :  -1837116.2
cost:  192580260.0
Iteration 1500
sum :  -1930873.2
cost:  160938400.0
Iteration 1600
sum :  -2020672.1
cost:  152959060.0
Iteration 1700
sum :  -2103264.0
cost:  145366820.0
Iteration 1800
sum :  -2182016.2
cost:  141120380.0
Iteration 1900
sum :  -2256179.0
cost:  141778480.0
######################

KeyboardInterrupt: 

In [25]:
tf.train.Saver.save??